In [581]:
import numpy as np
import decimal
import matplotlib.pyplot as plt

### Testing implementations MinHash 

In [284]:
data1 = np.array([0, 1, 1, 0, 0, 1, 0, 1])
data2 = np.array([0, 0, 1, 1, 0, 1, 1, 0])

overlap = np.sum(data1[np.where(data1 == data2)[0]])
rest = np.sum(data1[np.where(data1 != data2)[0]]) + np.sum(data2[np.where(data1 != data2)[0]])  

jaccard_similarity = float(overlap)/float(overlap + rest)
print("Actual jaccard similarity", jaccard_similarity)

Actual jaccard similarity 0.3333333333333333


In [377]:
max_hash_value = np.uint64((1 << 31) - 1)
def init_hash(verbose=False):
    seed = np.random.randint(1000)
    gen = np.random.RandomState(seed)
    c = np.uint64((1 << 31) - 1) # mersene_prime
    a = gen.randint(1, c, dtype=np.uint64)
    b = gen.randint(0, c, dtype=np.uint64)
    if verbose:
        print("a = {:d}, b = {:d}, c = {:d}".format(a,b,c))
    return a,b,c

def hash_value_01(val, params):
    a, b, c = params
    return float((a*val + b)%c)/float(max_hash_value)

In [282]:
for i in range(100000):
    params = init_hash()
    if hash_value_01(0, params) > 1:
        print("Oops! Value greater than 1", hash_value(0))
        break

In [294]:
k = 15
js = []
for i in range(100000):
    minhash1 = [] 
    minhash2 = []
    for i in range(k):
        params = init_hash()
        hash1 = []
        for j in np.where(data1 == 1)[0]:
            hash1.append(hash_value_01(j, params))

        hash2 = []
        for j in np.where(data2 == 1)[0]:
            hash2.append(hash_value_01(j, params))

        minhash1.append(min(hash1))
        minhash2.append(min(hash2))
    est_js = float(len(np.where(np.array(minhash1)==np.array(minhash2))[0]))/float(len(minhash1))
    js.append(est_js)
#     print("Estimated Jaccard Similarity = {:.4f}".format(est_js))
print("Mean of estimated JS = {:.4f}".format(np.mean(js)))
print("Std. dev of estimated JS = {:.4f}".format(np.std(js)))

Mean of estimated JS = 0.2991
Std. dev of estimated JS = 0.1182


### Implementations for Hash Functions - h, s, s_bar

In [445]:
def hash_value_h(data, losc_params, ind, verbose=False):
    assert(type(data) == np.ndarray)
    if verbose:
        print(losc_params.t_hash_h[ind], ind)
    r_hash = []
    for i in range(len(losc_params.r_hash_h)):
        params = losc_params.r_hash_h[i]
        hash_ = []
        
        for j in np.where(data == 1)[0]:
            hash_.append(hash_value_01(j, params))
            
        r_hash.append(min(hash_) if len(hash_) else 0.0)
        
    val = (np.sum(losc_params.h_weights*r_hash) + 13)*len(r_hash) 
    hash_val = hash_value_01(val, losc_params.t_hash_h[ind])
    if verbose:
        print(hash_val)
    return int(np.floor(hash_val*losc_params.b))%losc_params.b

def hash_value_s(data, losc_params, ind, verbose=False):
    assert(type(data) == np.ndarray)
    if verbose:
        print(losc_params.t_hash_s[ind], ind)
    val = (np.sum(losc_params.s_weights*data) + 23)*len(data) 
    hash_val = hash_value_01(val, losc_params.t_hash_s[ind])
    if verbose:
        print(hash_val)
    return 1 if hash_val >= 0.5 else -1

def hash_value_s_bar(data, losc_params, ind, verbose=False):
    assert(type(data) == np.ndarray)
    if verbose:
        print(losc_params.t_hash_h[ind], ind)
    p_hash = []
    for i in range(len(losc_params.l_hash_s_bar)):
        params = losc_params.l_hash_s_bar[i]
        hash_ = []
        for j in np.where(data == 1)[0]:
            hash_.append(hash_value_01(j, params))
        p_hash.append(min(hash_) if len(hash_) else 0.0)
 
    val = (np.sum(losc_params.s_bar_weights*p_hash) + 13)*len(p_hash) 
    hash_val = hash_value_01(val, losc_params.t_hash_s_bar[ind])
    if verbose:
        print(hash_val)
    return 1 if hash_val >= 0.5 else -1

In [323]:
test = np.array([0, 1, 1, 0, 0, 1, 0, 1])
sum_ = 0
r = 5
b = 10
r_weights = np.arange(1, r+1)
np.random.shuffle(r_weights)
bucket_cnt = [0]*b
for i in range(100000):
    params = init_hash(verbose=False)
    ind = hash_value_h(test, r, b, r_weights, params)
    bucket_cnt[ind] += 1
    
print("Mean, Std bucket count for hash function h = {:.5f}, {:.5f}".format(np.mean(bucket_cnt), np.std(bucket_cnt)))
print("Bucket indices filled", bucket_ind)

Mean, Std bucket count for hash function h = 10000.00000, 106.33532
Bucket indices filled [10142, 9995, 10018, 10065, 9753, 9849, 10002, 9858, 10094, 10224]


In [324]:
test = np.array([0, 1, 1, 0, 0, 1, 0, 1])
weights = np.arange(1, len(test)+1)
np.random.shuffle(weights)
sum_ = 0
for i in range(100000):
    params = init_hash(verbose=False)
    sum_ += hash_value_s(test, weights, params)
print("Error in hash function s = {:.5f}".format(abs(sum_)/100000))

Error in hash function s = 0.01268


In [328]:
test = np.array([0, 1, 1, 0, 0, 1, 0, 1])
sum_ = 0
l = 5
l_weights = np.arange(1, l+1)
np.random.shuffle(l_weights)
for i in range(100000):
    params = init_hash(verbose=False)
    sum_ += hash_value_s_bar(test, l, l_weights, params)
print("Error in hash function s_bar = {:.5f}".format(abs(sum_)/100000))

Error in hash function s_bar = 0.00556


### LoSC implementation

In [602]:
class LoSCParams:
    def __init__(self, t=10, b=10, r=5, l=5, d=10):
        self.b = b
        self.t = t
        self.r = r
        self.l = l
        self.d = d
        self.t_hash_h = []
        self.t_hash_s = []
        self.t_hash_s_bar = []
        self.r_hash_h = []
        self.l_hash_s_bar = []
        self.s_weights = []
        self.s_bar_weights = []
        self.h_weights = []
        
        self.initialize()
    
    def initialize(self):
        
        for i in range(self.t):
            self.t_hash_h.append(init_hash())
            self.t_hash_s.append(init_hash())
            self.t_hash_s_bar.append(init_hash())
            
        for i in range(self.r):
            self.r_hash_h.append(init_hash())
        
        for i in range(self.l):
            self.l_hash_s_bar.append(init_hash())
            
        self.s_weights = np.arange(1, self.d+1)
        np.random.shuffle(self.s_weights)
        
        self.s_bar_weights = np.arange(1, self.l+1)
        np.random.shuffle(self.s_bar_weights)
        
        self.h_weights = np.arange(1, self.r+1)
        np.random.shuffle(self.h_weights)

In [611]:
def add_losc(params): 
    loSC_css = [[0]*params.b for i in range(params.t)]
    loSC_sls = [[0]*params.b for i in range(params.t)]
    
    for item in dist2:
        for i in range(params.t):
            loSC_css[i][hash_value_h(item, params, i, False)] += hash_value_s(item, params, i)
            loSC_sls[i][hash_value_h(item, params, i, False)] += hash_value_s_bar(item, params, i)
            
    return loSC_css, loSC_sls

def estimate_losc(loSC_css, loSC_sls, query, params, verbose=False):
    estimate_css = []
    estimate_sls = []
    for i in range(params.t):
        estimate_css.append(loSC_css[i][hash_value_h(query, params, i, False)]*hash_value_s(query, params, i))
        estimate_sls.append(loSC_sls[i][hash_value_h(query, params, i, False)]*hash_value_s_bar(query, params, i))

    f_css = abs(np.median(estimate_css))
    f_sls = abs(np.median(estimate_sls))
    if verbose:
        print("Estimated frequency using LoSC-CSS = {:f}".format(f_css))
        print("Estimated frequency using LoSC-SLS = {:f}".format(f_sls))
    return f_css, f_sls

### Synthetic Data - Random Binary vectors 

In [626]:
dist2 = np.array([[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0]]
)

# there are 4 different items

# close group with 15/16 entries of 1s matching 
# [1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1]  14 copies
# [1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1]  16 copies

# close group with 6/8 entries of 1s matching
# [0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1]  16 copies
# [0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0]  16 copies

In [614]:
np.random.shuffle(dist2)

In [623]:
params = LoSCParams(t=10, b=15, r=3, l=3, d=len(dist2[0]))
loSC_css, loSC_sls = add_losc(params)

##### Estimating frequency of seen item

In [624]:
query = np.array([1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1])
estimate_losc(loSC_css, loSC_sls, query, params, verbose=True)

Estimated frequency using LoSC-CSS = 2.000000
Estimated frequency using LoSC-SLS = 30.000000


(2.0, 30.0)

##### Estimating frequency of unseen item

In [625]:
# a close item to [1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1]
query = np.array([1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]) 
estimate_losc(loSC_css, loSC_sls, query, params, verbose=True)

Estimated frequency using LoSC-CSS = 2.000000
Estimated frequency using LoSC-SLS = 30.000000


(2.0, 30.0)

Our LoSC-CSS approach has an LSH based implementation for the h function and since our sign function s is still a uniformly random function, the first 2 group of items end up with random sign (here opposite) and hence the final estimate is 2. 

However, for our LoSC-SLS implementation, since our sign function, s_bar, is an LSH based function which ends up mapping the first two group of items to the same sign and hence the final estimate is 30.